# Aggregate Functions 2

In [1]:
import pandas as pd 
import sqlite3


def create_df(cursor: sqlite3.Cursor) -> pd.DataFrame:
    data = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    df = pd.DataFrame(data, columns=column_names)
    return df


conn = sqlite3.connect(":memory:")

#### Prepare Database

In [2]:
st = """ 
    CREATE TABLE hacker_news (
        title TEXT,
        user TEXT,
        score INTEGER,
        timestamp DATETIME,
        url TEXT
    );
"""
conn.execute(st)

In [3]:
df = pd.read_csv("hacker_news.csv", delimiter=">")

title = df["title"].tolist()
user = df["user"].tolist()
score = df["score"].tolist()
timestamp = df["timestamp"].tolist()
url = df["url"].tolist()


In [4]:
conn.executemany(
    """INSERT INTO hacker_news (title, user, score, timestamp, url) 
        VALUES (?, ?, ?, ?, ?)
    """,
    zip(title, user, score, timestamp, url)
)

In [5]:
st = """ 
    SELECT * 
    FROM hacker_news;
"""

create_df(conn.execute(st)).head()

,title,user,score,timestamp,url
0,None,coldtea,NaN,2014-01-27T17:31:13Z,None
1,None,etanol,NaN,2011-10-23T18:46:40Z,None
2,None,None,NaN,2016-02-28T06:26:56Z,None
3,None,Locke1689,NaN,2014-08-12T22:13:10Z,None
4,None,miloshadzic,NaN,2013-03-06T12:28:02Z,None


#### 1.


Start by getting a feel for the `hacker_news` table!

Let’s find the most popular Hacker News stories:

```sql	
SELECT title, score
FROM hacker_news
ORDER BY score DESC
LIMIT 5;
```

What are the top five stories with the highest `score`s?

In [6]:
st = """ 
    SELECT title, score
    FROM hacker_news
    ORDER BY score DESC
    LIMIT 5;
"""
create_df(conn.execute(st))

,title,score
0,Penny Arcade – Surface Pro 3 update,517
1,Hacking The Status Game,309
2,Postgres CLI with autocompletion and syntax hi...,304
3,Stephen Fry hits out at ‘infantile’ culture of...,282
4,Reversal: Australian Govt picks ODF doc standa...,191


#### 2.

Recent studies have found that online forums tend to be dominated by a small percentage of their users ([1-9-90 Rule](https://en.wikipedia.org/wiki/1%25_rule)).

*Is this true of Hacker News?*

*Is a small percentage of Hacker News submitters taking the majority of the points?*

First, find the total `score` of all the stories.

In [7]:
st = """ 
    SELECT SUM(score) AS 'Total Score'
    FROM hacker_news;
"""
create_df(conn.execute(st))

,Total Score
0,6366


#### 3.

Next, we need to pinpoint the users who have accumulated a lot of points across their stories.

Find the individual users who have gotten combined `score`s of more than 200, and their combined `score`s.

`GROUP BY` and `HAVING` are needed!

In [8]:
st = """ 
    SELECT user, SUM(score) AS 'Total Score'
    FROM hacker_news
    GROUP BY user
    HAVING SUM(score) > 200;
"""
create_df(conn.execute(st))

,user,Total Score
0,amirkhella,309
1,dmmalam,304
2,metafunctor,282
3,vxNsr,517


#### 4.

Then, we want to add these users’ `score`s together and divide by the total to get the percentage.

Add their scores together and divide it by the total sum. Like so:

```sql
SELECT (1.0 + 2.0 + 3.0) / 6.0;
```

So, is Hacker News dominated by these users?

In [9]:
st = """ 
    SELECT ROUND(((517 + 309 + 304 + 282) / 6366.0), 3) * 100 AS 'Total Score in %'
"""
create_df(conn.execute(st))

,Total Score in %
0,22.2


#### 5. 

Oh no! While we are looking at the power users, some users are [rickrolling](https://knowyourmeme.com/memes/rickroll) — tricking readers into clicking on a link to a funny [video](https://www.youtube.com/watch?v=dQw4w9WgXcQ) and claiming that it links to information about coding.

The `url` of the video is:

`https://www.youtube.com/watch?v=dQw4w9WgXcQ`

*How many times has each offending user posted this link?*

In [10]:
st = """ 
    SELECT user, COUNT(*) AS 'Number of RickRolling URLs'
    FROM hacker_news
    WHERE url LIKE '%watch?v=dQw4w9WgXcQ%'
    GROUP BY 1
    ORDER BY 2 DESC;
"""
create_df(conn.execute(st))

,user,Number of RickRolling URLs
0,sonnynomnom,2
1,scorpiosister,1


#### 6.


Hacker News stories are essentially links that take users to other websites.

*Which of these sites feed Hacker News the most:*

*[GitHub](https://github.com/), [Medium](https://medium.com/), or [New York Times](https://www.nytimes.com/)?*

First, we want to categorize each story based on their source.

We can do this using a `CASE` statement:

```sql
SELECT CASE
   WHEN url LIKE '%github.com%' THEN 'GitHub'
   -- WHEN statement here
   -- WHEN statement here
   -- ELSE statement here
  END AS 'Source'
FROM hacker_news;
```

Fill in the other `WHEN` statements and the `ELSE` statement.

In [11]:
st = """ 
    SELECT CASE
        WHEN url LIKE '%github.com%' THEN 'GitHub'
        WHEN url LIKE '%medium.com%' THEN 'Medium'
        WHEN url LIKE '%nytimes.com%' THEN 'New York Times'
        ELSE 'Other'
        END AS 'Source'
    FROM hacker_news;
"""
create_df(conn.execute(st))

,Source
0,Other
1,Other
2,Other
3,Other
4,Other
...,...
3989,Other
3990,Other
3991,Other
3992,Other


#### 7.

Next, build on the previous query:

Add a column for the number of stories from each URL using `COUNT()`.

Also, `GROUP BY` the `CASE` statement.

Remember that you can refer to a column in `GROUP BY` using a number.

In [12]:
st = """ 
    SELECT CASE
        WHEN url LIKE '%github.com%' THEN 'GitHub'
        WHEN url LIKE '%medium.com%' THEN 'Medium'
        WHEN url LIKE '%nytimes.com%' THEN 'New York Times'
        ELSE 'Other'
        END AS 'Source', COUNT(*) AS 'Number of URLs'
    FROM hacker_news
    GROUP BY 1;
"""
create_df(conn.execute(st))

,Source,Number of URLs
0,GitHub,23
1,Medium,12
2,New York Times,13
3,Other,3946


#### 8.

Every submitter wants their story to get a high score so that the story makes it to the front page, but…

*What’s the best time of the day to post a story on Hacker News?*

Before we get started, let’s run this query and take a look at the `timestamp` column:

```sql
SELECT timestamp
FROM hacker_news
LIMIT 10;
```

Notice that the values are formatted like:

`2018-05-08T12:30:00Z`

If you ignore the `T` and `Z`, the format is:

`YYYY-MM-DD HH:MM:SS`

In [13]:
st = """ 
    SELECT timestamp
    FROM hacker_news
    LIMIT 10;
"""
create_df(conn.execute(st))

,timestamp
0,2014-01-27T17:31:13Z
1,2011-10-23T18:46:40Z
2,2016-02-28T06:26:56Z
3,2014-08-12T22:13:10Z
4,2013-03-06T12:28:02Z
5,2011-04-16T21:04:23Z
6,2014-03-18T21:44:46Z
7,2012-11-19T11:54:38Z
8,2016-11-04T13:55:30Z
9,2016-07-02T22:54:47Z


#### 9.

SQLite comes with a `strftime()` function - a very powerful function that allows you to return a formatted date.

It takes two arguments:

`strftime(format, column)`

Let’s test this function out:

```sql
SELECT timestamp,
   strftime('%H', timestamp)
FROM hacker_news
GROUP BY 1
LIMIT 20;
```

What do you think this does? Open the hint if you’d like to learn more.

<details><summary style="display:list-item; font-size:16px; color:white;">Hint</summary>

This returns the hour, `HH`, of the `timestamp` column!

For `strftime(__, timestamp)`:

- `%Y` returns the year (YYYY)
- `%m` returns the month (01-12)
- `%d` returns the day of the month (1-31)
- `%H` returns 24-hour clock (00-23)
- `%M` returns the minute (00-59)
- `%S` returns the seconds (00-59)

if `timestamp` format is `YYYY-MM-DD HH:MM:SS`.

Read more on the [SQLite documentation](https://www.sqlite.org/lang_datefunc.html).

In [ ]:
st = """ 
    SELECT timestamp, strftime('%H', timestamp)
    FROM hacker_news
    GROUP BY 1
    LIMIT 20;
"""
create_df(conn.execute(st))

,timestamp,"strftime('%H', timestamp)"
0,2007-03-16T20:52:19Z,20
1,2007-04-03T03:04:09Z,03
2,2007-05-01T03:11:17Z,03
3,2007-05-05T05:43:58Z,05
4,2007-05-11T05:48:53Z,05
5,2007-05-25T22:07:18Z,22
6,2007-06-08T08:44:50Z,08
7,2007-07-01T00:06:57Z,00
8,2007-07-27T16:47:00Z,16
9,2007-08-08T00:45:36Z,00


#### 10.


Okay, now we understand how `strftime()` works. Let’s write a query that returns three columns:

1. The hours of the `timestamp`
2. The average `score` for each hour
3. The count of stories for each hour

In [16]:
st = """ 
    SELECT strftime('%H', timestamp), AVG(score), COUNT(*)
    FROM hacker_news
    GROUP BY 1
    ORDER BY 2 DESC;
"""
create_df(conn.execute(st))

,"strftime('%H', timestamp)",AVG(score),COUNT(*)
0,18,27.027778,266
1,07,21.333333,104
2,19,20.413793,238
3,20,18.275862,239
4,12,14.310345,123
5,09,11.125000,119
6,16,9.755556,237
7,15,9.491228,268
8,17,8.860465,215
9,23,8.434783,170


#### 11.

Let’s edit a few things in the previous query:

- Round the average `score`s (`ROUND()`).
- Rename the columns to make it more readable (`AS`).
- Add a `WHERE` clause to filter out the `NULL` values in `timestamp`.

Take a look at the result again:

What are the best hours to post a story on Hacker News?

In [17]:
st = """ 
    SELECT strftime('%H', timestamp) AS 'Hour', ROUND(AVG(score), 1) AS 'Average Score', COUNT(*) AS 'Number of Stories'
    FROM hacker_news
    WHERE timestamp IS NOT NULL
    GROUP BY 1
    ORDER BY 2 DESC;
"""
create_df(conn.execute(st))

,Hour,Average Score,Number of Stories
0,18,27.0,266
1,07,21.3,104
2,19,20.4,238
3,20,18.3,239
4,12,14.3,123
5,09,11.1,119
6,16,9.8,237
7,15,9.5,268
8,17,8.9,215
9,23,8.4,170
